## Python to clips generator

In [15]:
import pandas as pd

In [16]:
df1 = pd.read_csv('llibres.csv')
df1.drop(columns='Unnamed: 0', inplace =True)
df1.drop(columns='Unnamed: 0.1', inplace =True)
df1.drop(columns='Unnamed: 0.2', inplace =True)
df1.drop(columns='Unnamed: 0.3', inplace =True)

df2 = pd.read_csv('authors_on_cluster.csv')
df2.drop(columns='Unnamed: 0', inplace =True)
df2.drop(columns='Unnamed: 0.1', inplace =True)

In [17]:
df2

,authors,Average Pages,Average Rating (0-5),Average Number of Characters,Average Minutes to Read,Average Weeks on Best-Seller List,Publishing_year_average,Most Common Genre,Most Common Subgenre,Most Common Writting Style,Cluster
0,A. E. van Vogt,418.0,3.90,65.00,710.6,36.00,1950.00,Science Fiction,space opera,Monólogo interior,7
1,Adolphe Napoléon Didron,453.0,3.30,64.00,770.1,28.00,1843.00,Historical,medieval,Estilo epistolar,4
2,Alan Dean Foster,246.0,2.20,54.00,418.2,9.00,2001.00,Science Fiction,space opera,Estilo experimental,7
3,Albert Camus,362.0,3.37,74.67,615.4,24.33,1933.67,Realistic,psychological,Narrativa en tercera persona,2
4,Alfred William Pollard,251.0,3.60,54.00,426.7,24.00,1890.00,Historical,medieval,Narrativa en primera persona,4
...,...,...,...,...,...,...,...,...,...,...,...
149,Virginia Woolf,282.0,4.80,70.00,479.4,36.00,1915.00,Adventure,travel,Estilo descriptivo,8
150,Walter Lippmann,284.0,3.00,53.00,482.8,21.00,1929.00,Realistic,social,Narrativa en tercera persona,2
151,Washington Irving,580.0,3.80,40.00,986.0,17.00,1800.00,Adventure,exploration,Estilo descriptivo,8
152,Wilhelmina Baird,391.0,3.50,98.00,664.7,1.00,1993.00,Science Fiction,cyberpunk,Estilo epistolar,0


In [18]:
# Create a dictionary of subgenres and subsubgenres
literary_genres = {
    "Realistic": ["social", "psychological", "manners"],
    "Historical": ["medieval", "ancient civilization", "world war"],
    "Romantic": ["contemporary romance", "historical romance", "paranormal romance"],
    "Science Fiction": ["space opera", "dystopian fiction", "cyberpunk"],
    "Fantasy": ["high fantasy", "low fantasy", "urban fantasy"],
    "Adventure": ["pirate", "travel", "exploration"]
}

#estilos = ['Narrativa en primera persona', 'Narrativa en tercera persona', 'Monólogo interior', 'Estilo experimental', 'Estilo epistolar', 'Estilo minimalista', 'Estilo descriptivo', 'Narrativa en segunda persona']


In [19]:
def conocimiento_a_clips(libros , autores, fichero = 'instances.clp', generos = {}):
    
    #Creación de diccionarios con valor identificativo para cada instancia.
    dicc_libros = {}
    for index, row in libros.iterrows():
        dicc_libros[row['title']] = f'Libro_Class_{index}'
    
    dicc_autores = {}
    for index, row in autores.iterrows():
        dicc_autores[row['authors']] = f'Autor_Class_{index}'
    
    dicc_generos = {}
    dicc_subgeneros = {}

    for index,value in enumerate(generos.keys()):
        dicc_generos[value] = f'Genero_Class_{index}'
    
        for index_2,value_2 in enumerate(generos[value]):
            dicc_subgeneros[value_2] = f'Subgenero_Class_{index*len(generos[value]) +index_2}'
        
    #Abrir un archivo en modo de escritura con codificación utf-8. Si el archivo no existe, será creado.
    with open(fichero, 'w', encoding='utf-8') as file:
        file.write('(definstances instances\n')
        
        #GENEROS
        for index, key in enumerate(generos.keys()):

            genero_instance = f"""
        ([{dicc_generos[key]}] of Genero
            (nombre "{key}")
        )"""

            # Escribir la instancia en el archivo
            file.write(genero_instance + '\n')
        
        #SUBGENEROS
        for key in generos.keys():
            for value in generos[key]:
                
                #Supergenero
                supergenero = dicc_generos[key]

                subgenero_instance = f"""
        ([{dicc_subgeneros[value]}] of Subgenero

            (nombre "{value}")
            (subgenero_de [{supergenero}])
        )"""
                # Escribir la instancia en el archivo
                file.write(subgenero_instance + '\n')
        
        #LIBROS
        for index, row in libros.iterrows():

            libro_instance = f"""
        ([{dicc_libros[row['title']]}] of Libro
            (genero_de [{dicc_generos[row['genre']]}])
            (subgenero_de [{dicc_subgeneros[row['subgenre']]}])
            (nombre "{row['title']}")
            (escrito_por [{dicc_autores[row['authors']]}])
            (publicacion {row['first_publish_year']})
            (valoracion {row['Rating (0-5)']})
            (paginas {row['Pages']})
            (popularidad {row['Weeks on Best-Seller List']})
            (complejidad {row['Number of Characters']})
            (estilo "{row['Estilo de Escritura']}")
            (publico "{row['Publico']}")
        )"""
            # Escribir la instancia en el archivo
            file.write(libro_instance + '\n')

        #AUTORES
        for index, row in autores.iterrows():
            # Lista de libros
            filtered_df = libros[libros['authors'] == row['authors']]
            libros_filtrados = filtered_df['title'].to_list()
            
            author_instance = f"""
        ([Autor_Class_{index}] of Autor
            (escribe_genero [{dicc_generos[row['Most Common Genre']]}])
            (escribe_subgenero [{dicc_subgeneros[row['Most Common Subgenre']]}])
            (media_paginas {row["Average Pages"]})
            (media_valoracion {row["Average Rating (0-5)"]})
            (complejidad {row["Average Number of Characters"]})
            (popularidad {row["Average Weeks on Best-Seller List"]}) 
            (media_fecha_publicacion {row["Publishing_year_average"]})
            (nombre "{row['authors']}")
            (cluster "{row['Cluster']}")
        )"""
            # Escribir la instancia en el archivo
            file.write(author_instance + '\n')
        
        # Escribir el final del bloque definstances
        file.write(')\n')


In [20]:
conocimiento_a_clips(libros = df1,autores = df2, generos = literary_genres)